In [5]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo
import sys

In [6]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


# Kucoin API object and Fake Wallet

In [7]:

kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
ORIGINAL_SYM_BIT = "BTC/USDT"
ORIGINAL_SYM_STR = "STRK/USDT"
symbol_btc = 'BTCUSDTM'
symbol_str = "STRKUSDTM"


SIZE_FOR_TRADE = 2
TARGET_GAIN = 1
TARGET_LOSS = 3



# Fake Wallet Class

In [36]:

class FakeWallet:
    def __init__(self, balance: float, coin_symbol: str) -> None:
        self.balance = balance
        self.coin_symbol = coin_symbol
        self.price_of_coin = 0
        self.side = "None"
        self.limit = balance
        self.date_of_creation = datetime.now()
        self.coin_quantity = 0

    def buy_coin(self, coin_price: float, coin_quantity: float, order: str) -> None:
        if order == "long":
            if coin_price * coin_quantity > self.balance:
                raise ValueError("Insufficient balance to buy the coin.")
            self.balance -= (coin_price * coin_quantity)
            self.coin_quantity += coin_quantity
            self.price_of_coin = coin_price
            self.side = "long"
        elif order == "short":
            self.side = "short"
            self.balance += (coin_price * coin_quantity)
            self.coin_quantity -= coin_quantity
            self.price_of_coin = coin_price
        else:
            raise ValueError(f"Order is not defined: {order}")

    def sell_coin(self, coin_price: float, coin_quantity: float, order: str) -> None:
        if order == "long":
            self.balance += (coin_price * coin_quantity)
            self.coin_quantity -= coin_quantity
            self.price_of_coin = coin_price
        elif order == "short":
            self.balance -= (coin_price * coin_quantity)
            self.coin_quantity += coin_quantity
        else:
            raise ValueError(f"Order is not defined: {order}")

    def get_pos_side(self) -> bool:
        if self.coin_quantity != 0:
            return True,self.side
        else:
            return False,None

    def status(self) -> None:
        print(f"Balance: {self.balance}")
        print(f"Coin Symbol: {self.coin_symbol}")
        print(f"Price of Coin: {self.price_of_coin}")
        print(f"Coin Quantity: {self.coin_quantity}")
        print(f"Side: {self.side}")
        print(f"Limit: {self.limit}")
        print(f"Date of Creation: {self.date_of_creation}")

F_wallet = FakeWallet(balance=200000,coin_symbol=ORIGINAL_SYM_BIT)

# Helping Function

In [9]:
from kucoin_futures.client import Trade

def get_size_pos(sy):        
    client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)
    try:
        positions = client.get_position_details(symbol=sy)
    except Exception as e:
        return 0,False
    return positions['currentQty'], positions['isOpen']

def ask_bid_kucoi(symbol):
    order_book = kucoin.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

def ask_bid_phemex(symbol):
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

# ! Helping Function for the Offline mode



# Daily SMA 20 Days

In [14]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=ORIGINAL_SYM_BIT, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as sell ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,60863.27,62234.38,60678.05,61699.91,182.290141,2024-06-27,NaN,NaN
1,61690.00,62199.64,60136.85,60419.36,207.377027,2024-06-28,NaN,NaN
2,60420.72,61183.98,60377.87,60984.93,121.646566,2024-06-29,NaN,NaN
3,60986.01,63000.00,60722.70,62773.92,157.579498,2024-06-30,NaN,NaN
4,62773.89,63797.82,62600.08,62896.97,192.767663,2024-07-01,NaN,NaN
...,...,...,...,...,...,...,...,...
95,65602.38,65613.99,62910.01,63301.97,211.430687,2024-09-30,62240.6515,SELL
96,63325.00,64093.23,60341.34,60852.45,203.211621,2024-10-01,62416.1795,SELL
97,60818.45,62290.69,60025.00,60628.99,239.408284,2024-10-02,62542.2930,SELL
98,60704.65,61438.43,59950.00,60733.86,221.541395,2024-10-03,62554.1110,SELL


#  15 Minutes SMA

In [18]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = kucoin.fetch_ohlcv(symbol=ORIGINAL_SYM_BIT, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()




starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,61584.2,61598.9,61445.0,61445.5,9.173682,2024-10-04 08:45:00,NaN,NaN,NaN,NaN,NaN
1,61445.5,61559.9,61419.8,61517.3,15.905319,2024-10-04 09:00:00,NaN,NaN,NaN,NaN,NaN
2,61517.3,61586.5,61379.3,61403.1,9.625051,2024-10-04 09:15:00,NaN,NaN,NaN,NaN,NaN
3,61403.0,61438.5,61327.1,61353.9,13.318135,2024-10-04 09:30:00,NaN,NaN,NaN,NaN,NaN
4,61353.9,61383.9,61306.2,61338.5,19.068403,2024-10-04 09:45:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,62151.0,62173.4,62087.2,62089.4,3.551532,2024-10-05 08:30:00,62072.410,62134.482410,61886.192770,62010.337590,62258.627230
96,62089.4,62156.4,62089.4,62156.4,1.844223,2024-10-05 08:45:00,62087.155,62149.242155,61900.893535,62025.067845,62273.416465
97,62156.4,62217.2,62145.3,62217.1,3.347283,2024-10-05 09:00:00,62106.805,62168.911805,61920.484585,62044.698195,62293.125415
98,62217.2,62277.9,62148.8,62229.1,16.669035,2024-10-05 09:15:00,62120.835,62182.955835,61934.472495,62058.714165,62307.197505


# Kill Switch & Pnl For Online

In [9]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pnl_c, _ ,side = pnl_close_kucoin() #! pnl_close, open_pos, Side-long,short
    size_for_kill,open_p = get_size_pos(sy=symbol_str)

    print(f"open position: {open_p} side: {side} Size postion: {size_for_kill}")

    while open_p:

        print("Starting the Kill Swtiching the Position...")
        # TODO: Code for cancling all the position

        open_p,size_for_kill= get_size_pos(sy=symbol_str)
        side = pnl_close_kucoin()[2]

        ask,bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_STR)

        if side == "long":
            
            # TODO: We are in long and Create SELL order
            print(f" ---- ----- ----- Just Made the Order for SELL at {ask} for {symbol_str} size: {size_for_kill}")
            print("30 secs After Order ...")
            time.sleep(30)

        elif side == "short":

            # TODO: We are in long and Create SELL order
            print(f" ---- ----- Just Made the Order for BUY at {bid} for {symbol_str} siz: {size_for_kill}")
            print("30 secs After Order ...")
            time.sleep(30)

        else:
            print("== == == Side is not defined so wait ...")
        
        size_for_kill, open_p = get_size_pos(sy=symbol_str)
        
    
def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    if (diff > 0 and current_price > entry_price) or ( diff < 0 and current_price < entry_price):
        side = "long"
    elif diff == 0:
        side = "None"
    else:
        side = "short"
    
    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but   not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()

Starting the Kill switching ...
Starting PNL ...
Difference: 0.01769, Current Price: 0.38663, Entry Price: 0.36894, Side: long, Percentage: 4.79%
In Profit but   not hit the target, so wait !!
Finishing the PNL close ...
open position: True side: long Size postion: 2
Starting the Kill Swtiching the Position...
Starting PNL ...
Difference: 0.01769, Current Price: 0.38663, Entry Price: 0.36894, Side: long, Percentage: 4.79%
In Profit but   not hit the target, so wait !!
Finishing the PNL close ...
 ---- ----- ----- Just Made the Order for SELL at 0.38608 for STRKUSDTM size: True
30 secs After Order ...


KeyboardInterrupt: 

# Offline setup

In [37]:
from aifc import Error


def gain_position(cur_price):
        print("Checking the Profit and Loss status")
        pos, side = F_wallet.get_pos_side()
        if not pos:
            return print("Not in Position")
        if side == "long":
            return ((cur_price - float(F_wallet.price_of_coin)) / float(F_wallet.price_of_coin)) * 100
        elif side == "short":
            return ((float(F_wallet.price_of_coin) - cur_price) / float(F_wallet.price_of_coin)) * 100
        else:
            return print("Side is not Considered")
        
def kill_position(cur):
    pos,side = F_wallet.get_pos_side()
    while pos:
        print(f"Start Killing {side} Positions ...")
        if side == "long":
            F_wallet.sell_coin(cur,SIZE_FOR_TRADE,"long")
            time.sleep(5)
        else:
            F_wallet.sell_coin(cur,SIZE_FOR_TRADE,"short")
            time.sleep(5)

        pos,side = F_wallet.get_pos_side()
                 
    print("Killing Position Completed ...")



In [42]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask, bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)

    # * Define OPEN Order
    # signal = data_20d.iloc[-1]['SIGNAL']
    
    signal = "BUY"
    #! For Online
    # pnl_close_kucoin()
    # _,in_pos = get_size_pos(sy=ORIGINAL_SYM_BIT)

    # *OFFLINE
    in_pos, side = F_wallet.get_pos_side()

    if not in_pos:
        print(f"Signal For the Trade: {signal}")
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...

            avg = (bp_1 + bp_2) / 2

            # *OFFLINE - FAKE WALLET
            F_wallet.buy_coin(avg, SIZE_FOR_TRADE, "long")

            print("Wait for 10 sec After Order ...")
            time.sleep(10)
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ...

            # *OFFLINE - FAKE WALLET
            avg = (sp_1 + sp_2) / 2

            F_wallet.buy_coin(avg, SIZE_FOR_TRADE, "short")
            print("Wait for 10 sec After Order ...")
            time.sleep(10)
    else:
        cur_price = ask if side == "short" else bid  # moved assignment up
        print(f"Already In Position, not making new Position")
        print(f"Current Price is {cur_price}")
        gain = gain_position(cur_price=float(cur_price))
        if gain and gain >= TARGET_GAIN:
            print(f"Achieved the Target with gain {gain} Hooorahhh !!")
            kill_position(cur=float(cur_price))
        elif gain:
            print(f"Gain is {gain}, but below target.")  # give meaningful print statement here

bot()


starting sma 15 Minuts  ...
starting 20 Days ...
Opening as sell ...
Already In Position, not making new Position
Current Price is 62224.3
Checking the Profit and Loss status
Gain is 0.24546369904077195, but below target.


In [45]:
F_wallet.status()

Balance: 75856.12813999999
Coin Symbol: BTC/USDT
Price of Coin: 62071.93593000001
Coin Quantity: 2
Side: long
Limit: 200000
Date of Creation: 2024-10-05 15:08:30.070019
